In [1]:
from os.path import (
    abspath,
    dirname,
    exists,
    join,
)
from pandas import (
    read_csv,
    merge,
    concat,
    DataFrame,
    Series,
)
from sklearn.model_selection import train_test_split

In [2]:
PWD = (abspath(''))
datadir = join(PWD, '../Datasets/ml-1m/')
files = {}
files['train'] = join(datadir, 'train.csv')
files['test'] = join(datadir, 'test.csv')
THRESHOLD = 200
TAKE_TOP = 10

In [3]:
dfs = {}
for file in files:
    dfs[file] = read_csv(
        files[file],
        engine='python',
        encoding='latin1',
    )

In [4]:
dfs['train'].head()

,title,rating,userId
0,Airplane! (1980),0.8,1
1,Aladdin (1992),0.8,1
2,Antz (1998),0.8,1
3,Apollo 13 (1995),1.0,1
4,Awakenings (1990),1.0,1


In [5]:
dfs['test'].head()

,title,rating,userId
0,Snow White and the Seven Dwarfs (1937),0.8,1
1,"Sound of Music, The (1965)",1.0,1
2,Star Wars: Episode IV - A New Hope (1977),0.8,1
3,Tarzan (1999),0.6,1
4,Titanic (1997),0.8,1


In [6]:
dfs['train'].shape, dfs['test'].shape

((755865, 3), (99865, 3))

In [7]:
dfs['test'][dfs['test']['title'].isin(dfs['train']['title'])].shape

(99865, 3)

In [8]:
dfs['pivot'] = dfs['train'].pivot_table(
    index=['userId'],
    columns=['title'],
    values='rating',
)

In [9]:
dfs['pivot'].head()

title,"'burbs, The (1989)",10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),2010 (1984),...,Year of Living Dangerously (1982),Yellow Submarine (1968),Yojimbo (1961),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Zero Effect (1998),eXistenZ (1999)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,0.8,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
dfs['pivot'].shape

(6040, 1426)

In [11]:
corr_movies = dfs['pivot'].corr(method='pearson', min_periods=THRESHOLD)

In [12]:
corr_movies.head()

title,"'burbs, The (1989)",10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),2010 (1984),...,Year of Living Dangerously (1982),Yellow Submarine (1968),Yojimbo (1961),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Zero Effect (1998),eXistenZ (1999)
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10 Things I Hate About You (1999),NaN,1.0,NaN,NaN,NaN,0.266755,NaN,NaN,-0.068404,NaN,...,NaN,NaN,NaN,0.194905,NaN,NaN,NaN,NaN,NaN,NaN
101 Dalmatians (1961),NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.069166,NaN,...,NaN,NaN,NaN,NaN,0.249222,NaN,NaN,NaN,NaN,NaN
101 Dalmatians (1996),NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12 Angry Men (1957),NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.102686,NaN,...,NaN,NaN,NaN,NaN,0.249671,NaN,NaN,NaN,NaN,NaN


In [13]:
corr_movies += 1.0
corr_movies /= 2.0

In [14]:
corr_movies.head()

title,"'burbs, The (1989)",10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),2010 (1984),...,Year of Living Dangerously (1982),Yellow Submarine (1968),Yojimbo (1961),You've Got Mail (1998),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),Young Sherlock Holmes (1985),Zero Effect (1998),eXistenZ (1999)
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10 Things I Hate About You (1999),NaN,1.0,NaN,NaN,NaN,0.633378,NaN,NaN,0.465798,NaN,...,NaN,NaN,NaN,0.597452,NaN,NaN,NaN,NaN,NaN,NaN
101 Dalmatians (1961),NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.534583,NaN,...,NaN,NaN,NaN,NaN,0.624611,NaN,NaN,NaN,NaN,NaN
101 Dalmatians (1996),NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12 Angry Men (1957),NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.551343,NaN,...,NaN,NaN,NaN,NaN,0.624835,NaN,NaN,NaN,NaN,NaN


In [15]:
dfs['users'] = dfs['train'].pivot_table(
    index=['title'],
    columns=['userId'],
    values='rating',
)

In [16]:
dfs['users'].head()

userId,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10 Things I Hate About You (1999),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.4,NaN,NaN,NaN,NaN
101 Dalmatians (1961),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.8,NaN,NaN,NaN,0.8,NaN,NaN,NaN,NaN
101 Dalmatians (1996),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.2,NaN,NaN,NaN,NaN,NaN
12 Angry Men (1957),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6,...,NaN,NaN,NaN,NaN,NaN,NaN,0.8,NaN,NaN,1.0


In [17]:
corr_user = dfs['users'].corr(method='pearson', min_periods=THRESHOLD)

In [18]:
corr_user += 1.0
corr_user /= 2.0

In [19]:
corr_user.head()

userId,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
dfs['test'] = dfs['test'].sort_values(
    by=['userId','title','rating'],
    ascending=False,
)

In [21]:
result = dfs['test'].pivot_table(
    index=['userId'],
    columns=['title'],
    values='rating',
)

In [22]:
result.head()

title,Aliens (1986),AlienÂ³ (1992),All Dogs Go to Heaven (1989),All Quiet on the Western Front (1930),Amadeus (1984),American Beauty (1999),"American President, The (1995)","American Werewolf in London, An (1981)",Amistad (1997),"Amityville Horror, The (1979)",...,To Kill a Mockingbird (1962),Tombstone (1993),Top Hat (1935),Total Recall (1990),Toy Story (1995),Toy Story 2 (1999),True Grit (1969),U-571 (2000),Unforgiven (1992),Wayne's World (1992)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.8,NaN,NaN,NaN,1.0,0.8,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.8,NaN,NaN,0.6,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.6,NaN,NaN,1.0,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.2,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
users_test = result.index
output = DataFrame()
for user in users_test[:]:
    recommendations = DataFrame()
    ratings = dfs['pivot'].loc[user].dropna()
    ratings = ratings.sort_values(ascending=False)
    #print(ratings.head())
    for movie in ratings.index:
        movies_sim = corr_movies[movie].dropna()
        movies_sim = DataFrame({
            'title': movies_sim.index,
            'rating': movies_sim.values,
        })
        movies_sim = movies_sim.sort_values(by=['rating'],ascending=False)
        movies_sim = movies_sim.reset_index(drop=True)
        #movies_sim = movies_sim[:TAKE_TOP]
        movies_sim['rating'] *= ratings[movie]
        #print(movie)
        #print(movies_sim.head())
        recommendations = concat([recommendations, movies_sim], ignore_index=True)
    #print(recommendations.head())
    recommendations = recommendations.groupby(by=['title'], as_index=False).agg(lambda x: sum(x))
    recommendations = recommendations.sort_values(by='rating',ascending=False)
    recommendations = recommendations[~recommendations['title'].isin(ratings.index)]
    recommendations = recommendations.reset_index(drop=True)
    recommendations = recommendations[:TAKE_TOP]
    recommendations['userId'] = user
    output = concat([output, recommendations], ignore_index=True)
    #print(user)
    #print('recommended')
    #print(recommendations.head())
    movies_actual = result.loc[user].dropna()
    movies_actual = DataFrame({
        'title': movies_actual.index,
        'rating': movies_actual.values,
    })
    movies_actual = movies_actual.sort_values(
        by=['rating', 'title'],
        ascending=False,
    )
    #print('Actual')
    #print(movies_actual.head())

In [24]:
output.head()

,title,rating,userId
0,Forrest Gump (1994),20.695031,1
1,Babe (1995),20.307852,1
2,Jurassic Park (1993),20.182707,1
3,"Fugitive, The (1993)",20.122368,1
4,Raiders of the Lost Ark (1981),19.887378,1


In [25]:
output.to_csv(join(datadir, 'output.csv'), index=False)